In [ ]:
import numpy as np

# 先运行并载入 basis_func_with_kinetic.ipynb 中定义的函数（primitive_fcc, reciprocal_vectors, generate_G_vectors 等）
%run basis_func_with_kinetic.ipynb

# 说明：
# 1) 给出周期体系中 Hartree 能的常用表达式（原子单位）：
#    E_H = (2*pi / Omega) * sum_{G != 0} |n(G)|^2 / |G|^2
#    其中 n(G) = (1/Omega) * integral_{cell} n(r) e^{-i G·r} dr
# 2) 本示例展示如何：
#    - 在实空间用格点构造一个归一化的 Gaussian 电荷密度 n(r) (单位: e/bohr^3)
#    - 用 FFT 计算其 n(G)（按 n(G)=1/Omega * integral 的离散近似）
#    - 用上面的公式计算 Hartree 能（Hartree 单位），并输出 eV 值

HARTREE_TO_EV = 27.211386

def compute_hartree_energy_from_nG(nG, Gvecs, volume):
    """根据 n(G) 和对应的 G 向量计算 Hartree 能（原子单位：Hartree）。
    nG: ndarray，复数，形状 (Ngrid,) 或 (Nx,Ny,Nz) 扁平或网格形式，表示 n(G)=1/Ω ∫ n(r)e^{-iG·r} dr 的近似值
    Gvecs: ndarray，形状与 nG 对应的 (N,3) 或 (Nx,Ny,Nz,3)，单位 1/bohr
    volume: 单胞体积 Omega，单位 bohr^3
    返回: E_H (Hartree)
    """
    nG_flat = nG.ravel()
    G_flat = Gvecs.reshape((-1, 3))

    norms = np.linalg.norm(G_flat, axis=1)
    mask = norms > 1e-12
    G_nonzero = G_flat[mask]
    nG_nonzero = nG_flat[mask]

    prefac = 2.0 * np.pi / volume
    E_H = prefac * np.sum(np.abs(nG_nonzero)**2 / (np.linalg.norm(G_nonzero, axis=1)**2))
    return E_H

def build_gaussian_density(a0=4.05, Nk=32, sigma_bohr=1.0):
    """
    构造 FCC 晶格下的 Gaussian 电子密度并归一化到一个电子。
    返回: n_r (密度网格), Omega (单胞体积), Nk, a_vecs, 以及 r 网格坐标。
    """
    a_vecs = primitive_fcc(a0)
    Omega = np.dot(a_vecs[0], np.cross(a_vecs[1], a_vecs[2]))

    us = np.arange(Nk) / Nk
    U, V, W = np.meshgrid(us, us, us, indexing='ij')
    r = (U[...,None] * a_vecs[0] + V[...,None] * a_vecs[1] + W[...,None] * a_vecs[2])

    r_center = 0.5 * (a_vecs[0] + a_vecs[1] + a_vecs[2])
    rvecs = r - r_center
    r2 = np.sum(rvecs**2, axis=3)

    pref = (1.0 / (sigma_bohr * np.sqrt(np.pi)))**3
    n_r = pref * np.exp(-r2 / (sigma_bohr**2))
    dV = Omega / (Nk**3)
    n_r /= np.sum(n_r) * dV

    return n_r, Omega, Nk, a_vecs, r

def density_grid_to_reciprocal(n_r, b_vecs):
    """从实空间密度网格生成 n(G) 与对应的 G 网格，便于后续能量计算。"""
    if n_r.ndim != 3 or n_r.shape[0] != n_r.shape[1] or n_r.shape[1] != n_r.shape[2]:
        raise ValueError('n_r must be a cubic grid with shape (Nk, Nk, Nk).')
    Nk = n_r.shape[0]
    nG_fft = np.fft.fftn(n_r)
    nG = nG_fft / nG_fft.size
    freq = np.fft.fftfreq(Nk) * Nk
    mx, my, mz = np.meshgrid(freq, freq, freq, indexing='ij')
    Ggrid = (mx[..., None] * b_vecs[0] + my[..., None] * b_vecs[1] + mz[..., None] * b_vecs[2]) / Nk
    return nG, Ggrid

# ---------------- 示例: 从实空间 Gaussian 密度用 FFT 求 n(G) 并计算 Hartree 能 ----------------
# if __name__ == '__main__':
#     # 使用统一的构造函数生成密度及晶格对象
#     a0 = 4.05  # Å (Al)
#     Nk = 32
#     sigma_bohr = 1.0
#     n_r, Omega, Nk, a_vecs, r = build_gaussian_density(a0=a0, Nk=Nk, sigma_bohr=sigma_bohr)
#     b_vecs, _ = reciprocal_vectors(a_vecs)
#     print(f'Cell volume (bohr^3): {Omega:.6f}')

#     # 实空间 -> 倒空间
#     nG, Ggrid = density_grid_to_reciprocal(n_r, b_vecs)

#     E_H_ha = compute_hartree_energy_from_nG(nG, Ggrid, Omega)
#     print(f'Hartree energy (Hartree): {E_H_ha:.8f}')
#     print(f'Hartree energy (eV): {E_H_ha * HARTREE_TO_EV:.6f} eV')

#     # 为了可视化：打印前 8 非零 G 的 |n(G)|^2/|G|^2 值 
#     nG_flat = nG.ravel()
#     G_flat = Ggrid.reshape((-1,3))
#     norms = np.linalg.norm(G_flat, axis=1)
#     mask = norms > 1e-12
#     vals = np.abs(nG_flat[mask])**2 / (norms[mask]**2)
#     order = np.argsort(vals)[::-1]
#     for i in range(min(8, len(order))):
#         idx = order[i]
#         print(f'Largest contrib #{i+1}: |n(G)|^2/|G|^2 = {vals[idx]:.6e}, |G| = {norms[mask][idx]:.6e}')



Crystal: Al (FCC), a = 4.05 Å
Ecut = 300.0 eV -> 11.02480 Hartree
Number of plane waves (Gamma): 181
First 8 G coefficients: [(0, 0, 0), (1, 1, 1), (1, 0, 0), (0, 1, 0), (0, 0, 1), (0, 0, -1), (-1, 0, 0), (-1, -1, -1)]
First 8 kinetic energies (Hartree): [0.         1.01098166 1.01098166 1.01098166 1.01098166 1.01098166
 1.01098166 1.01098166]
Cell volume (bohr^3): 112.073152
Hartree energy (Hartree): 0.01052706
Hartree energy (eV): 0.286456 eV
Largest contrib #1: |n(G)|^2/|G|^2 = 1.486583e-02, |G| = 4.443617e-02
Largest contrib #2: |n(G)|^2/|G|^2 = 1.486583e-02, |G| = 4.443617e-02
Largest contrib #3: |n(G)|^2/|G|^2 = 1.486583e-02, |G| = 4.443617e-02
Largest contrib #4: |n(G)|^2/|G|^2 = 1.486583e-02, |G| = 4.443617e-02
Largest contrib #5: |n(G)|^2/|G|^2 = 1.486583e-02, |G| = 4.443617e-02
Largest contrib #6: |n(G)|^2/|G|^2 = 1.486583e-02, |G| = 4.443617e-02
Largest contrib #7: |n(G)|^2/|G|^2 = 1.476170e-02, |G| = 4.443617e-02
Largest contrib #8: |n(G)|^2/|G|^2 = 1.476170e-02, |G| = 4.44